# LAB 3
- Edwin Montenegro
- Galo Travez

# NLP and Neural Networks

In this exercise, we'll apply our knowledge of neural networks to process natural language. As we did in the bigram exercise, the goal of this lab is to predict the next word, given the previous one.

### Data set

Load the text from "One Hundred Years of Solitude" that we used in our bigrams exercise. It's located in the data folder.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
text = open('/content/drive/MyDrive/TALLERES NLP /cap_1_first_10.txt', 'r').read().lower()

### Important note:

Start with a smaller part of the text. Maybe the first 10 parragraphs, as the number of tokens rapidly increases as we add more text.

Later you can use a bigger corpus.

In [3]:
text

'muchos años después, frente al pelotón de fusilamiento, el coronel aureliano buendía había de recordar aquella tarde remota en que su padre lo llevó a conocer el hielo. macondo era entonces una aldea de veinte casas de barro y cañabrava construidas a la orilla de un río de aguas diáfanas que se precipitaban por un lecho de piedras pulidas, blancas y enormes como huevos prehistóricos. el mundo era tan reciente, que muchas cosas carecían de nombre, y para mencionarlas había que señalarlas con el dedo. todos los años, por el mes de marzo, una familia de gitanos desarrapados plantaba su carpa cerca de la aldea, y con un grande alboroto de pitos y timbales daban a conocer los nuevos inventos. primero llevaron el imán. un gitano corpulento, de barba montaraz y manos de gorrión, que se presentó con el nombre de melquíades, hizo una truculenta demostración pública de lo que él mismo llamaba la octava maravilla de los sabios alquimistas de macedonia. fue de casa en casa arrastrando dos lingote

Don't forget to prepare the data by generating the corresponding tokens.

In [4]:
import torch
from nltk.tokenize import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()

In [5]:
tokens = tokenizer.tokenize(text)

In [6]:
tokens[:50]

['muchos',
 'años',
 'después',
 ',',
 'frente',
 'al',
 'pelotón',
 'de',
 'fusilamiento',
 ',',
 'el',
 'coronel',
 'aureliano',
 'buendía',
 'había',
 'de',
 'recordar',
 'aquella',
 'tarde',
 'remota',
 'en',
 'que',
 'su',
 'padre',
 'lo',
 'llevó',
 'a',
 'conocer',
 'el',
 'hielo.',
 'macondo',
 'era',
 'entonces',
 'una',
 'aldea',
 'de',
 'veinte',
 'casas',
 'de',
 'barro',
 'y',
 'cañabrava',
 'construidas',
 'a',
 'la',
 'orilla',
 'de',
 'un',
 'río',
 'de']

### Let's prepare the data set.

Our neural network needs to have an input X and an output y. Remember that these sets are numerical, so you'd need something to map the tokens into numbers, and viceversa.

In [8]:
# in this case, let's consider a bigram (w1, w2)
# assign the w1 to the X vector, and w2 to the y vector, why do we do this?

In [7]:
# Crear un conjunto único de palabras para el vocabulario
vocab = set(tokens)

# Mapear cada palabra a un índice único
word_to_idx = {word: i for i, word in enumerate(vocab)}
idx_to_word = {i: word for word, i in word_to_idx.items()}

# Convertir los tokens a índices numéricos utilizando el diccionario creado
indices = [word_to_idx[word] for word in tokens]

# Verificar los primeros índices
print(indices[:50])  # Mostrar los primeros 50 índices


[675, 513, 639, 598, 15, 901, 769, 666, 1038, 598, 541, 392, 1158, 1266, 1042, 666, 758, 1257, 189, 1016, 1233, 607, 1043, 751, 812, 892, 1163, 339, 541, 586, 525, 344, 625, 554, 765, 666, 1247, 45, 666, 324, 1252, 668, 818, 1163, 378, 1217, 666, 1087, 1133, 666]


In [8]:
vocab_size = len(vocab)
vocab_size


1367

In [9]:
import torch

# Crear bigramas (w1, w2)
bigrams = [(indices[i], indices[i+1]) for i in range(len(indices) - 1)]

# Separar los bigramas en X e y
X, y = zip(*bigrams)

# Convertir X e y a tensores
X = torch.tensor(X, dtype=torch.long)
y = torch.tensor(y, dtype=torch.long)

print(X[:5], y[:5])  # Mostrar los primeros 5 ejemplos


tensor([675, 513, 639, 598,  15]) tensor([513, 639, 598,  15, 901])


In [12]:
# Don't forget that since we are using torch, our training set vectors should be tensors

In [13]:
# Note that our vectors are integers, which can be thought as a categorical variables.
# torch provides the one_hot method, that would generate tensors suitable for our nn
# make sure that the dtype of your tensor is float.

In [10]:
# Definir la función de one-hot encoding
def one_hot_encode(indices, vocab_size):
    return torch.eye(vocab_size)[indices]

# Tamaño del vocabulario
vocab_size = len(vocab)

# Aplicar one-hot encoding a X
X_one_hot = one_hot_encode(X, vocab_size)

print(X_one_hot.shape)  # Verificar la forma de los datos


torch.Size([3602, 1367])


### Network design
To start, we are going to have a very simple network. Define a single layer network

In [28]:
import torch.nn as nn
import torch.optim as optim

class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)  # Convertir índices de palabras a vectores de embedding
        lstm_out, _ = self.lstm(x)  # Pasar por la LSTM
        out = self.fc(lstm_out)  # Pasar por la capa de salida para obtener logits
        return torch.softmax(out, dim=-1)  # Aplicar softmax para obtener probabilidades


In [29]:
# How many neurons should our input layer have?
# Use as many neurons as the total number of categories (from your one-hot encoded tensors)

# Configurar dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Instanciar el modelo y moverlo a GPU
model = LSTMModel(vocab_size=vocab_size, embedding_dim=50, hidden_dim=100).to(device)

# Definir la función de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Convertir X e y a tensores y moverlos a GPU
X_tensor = torch.tensor(indices[:-1], dtype=torch.long).to(device)
y_tensor = torch.tensor(indices[1:], dtype=torch.long).to(device)

# Entrenamiento
epochs = 400
for epoch in range(epochs):
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_tensor.unsqueeze(1))

    # Calcular pérdida
    loss = criterion(outputs.squeeze(1), y_tensor)

    # Backward pass y optimización
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 50 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [50/400], Loss: 7.2191
Epoch [100/400], Loss: 7.1732
Epoch [150/400], Loss: 7.1029
Epoch [200/400], Loss: 7.0505
Epoch [250/400], Loss: 7.0188
Epoch [300/400], Loss: 6.9909
Epoch [350/400], Loss: 6.9674
Epoch [400/400], Loss: 6.9419


In [17]:
# Use the softmax as your activation layer

In [30]:
# Train your network
def generar_texto(word, model, word_to_idx, idx_to_word, max_length=50):
    model.eval()
    g = torch.Generator().manual_seed(67)  # Semilla para reproducibilidad
    sentence = [word]
    ix = word_to_idx[word]

    for _ in range(max_length - 1):
        input_seq = torch.tensor([ix], dtype=torch.long).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(input_seq)

        # Muestrear la siguiente palabra basada en las probabilidades del modelo
        p = output.squeeze().cpu()
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        word = idx_to_word[ix]
        sentence.append(word)

        # Romper si la oración termina con un punto
        if '.' in word:
            break

    return ' '.join(sentence)

### Analysis

1. Test your network with a few words

In [33]:
# Probar la generación de texto con algunas palabras iniciales
palabras_iniciales = ['aureliano', 'macondo', 'muchos', 'orilla']
for i, palabra in enumerate(palabras_iniciales):
    oracion_generada = generar_texto(palabra, model, word_to_idx, idx_to_word, max_length=20)
    print(f"{i} {oracion_generada}")

0 aureliano , y el mundo albahaca.
1 macondo , y el mundo albahaca.
2 muchos exacto negro , y el mundo comprobado calor.
3 orilla de la aldea de la aldea , y el mundo derretida por el mundo peste , y el mundo


In [34]:
# Calcular la log-verosimilitud negativa para un conjunto de tokens
def calcular_nll(tokens, model, word_to_idx):
    model.eval()
    log_likelihood = 0.0
    n = 0

    for w1, w2 in zip(tokens, tokens[1:]):
        ix1 = word_to_idx[w1]
        ix2 = word_to_idx[w2]

        input_seq = torch.tensor([ix1], dtype=torch.long).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(input_seq).squeeze()

        pr = output[ix2]
        log_likelihood += torch.log(pr).item()
        n += 1

    return log_likelihood, -log_likelihood, -log_likelihood / n if n > 0 else float('inf')

# Calcular NLL para algunas oraciones generadas
for i, palabra in enumerate(palabras_iniciales):
    oracion_generada = generar_texto(palabra, model, word_to_idx, idx_to_word, max_length=20)
    tokens_generados = oracion_generada.split()
    log_likelihood, nll, nnll = calcular_nll(tokens_generados, model, word_to_idx)
    print(f"{i} Oración: '{oracion_generada}'\nLog-Likelihood: {log_likelihood:.4f}, NLL: {nll:.4f}, Normalized NLL: {nnll:.4f}\n")

0 Oración: 'aureliano , y el mundo albahaca.'
Log-Likelihood: -8.0678, NLL: 8.0678, Normalized NLL: 1.6136

1 Oración: 'macondo , y el mundo albahaca.'
Log-Likelihood: -8.0496, NLL: 8.0496, Normalized NLL: 1.6099

2 Oración: 'muchos exacto negro , y el mundo comprobado calor.'
Log-Likelihood: -27.4759, NLL: 27.4759, Normalized NLL: 3.4345

3 Oración: 'orilla de la aldea de la aldea , y el mundo derretida por el mundo peste , y el mundo'
Log-Likelihood: -21.1965, NLL: 21.1965, Normalized NLL: 1.1156



2. What does each value in the tensor represents?
3. Why does it make sense to choose that number of neurons in our layer?
4. What's the negative likelihood for each example?
5. Try generating a few sentences?
6. What's the negative likelihood for each sentence?


2. Cada valor en el tensor de salida del modelo representa la probabilidad predicha de que una palabra específica en el vocabulario sea la siguiente en la secuencia, dado el contexto actual de las palabras anteriores.
3. Tiene sentido elegir un número de neuronas en la capa de salida igual al tamaño del vocabulario (vocab_size) porque estamos intentando predecir la próxima palabra en la secuencia entre todas las palabras posibles en nuestro vocabulario
4. Una NLL(negative likelihood) más baja indica que el modelo predijo con más confianza y precisión la próxima palabra, mientras que una NLL más alta indica menos confianza y precisión en la predicción.
6. En las Oraciones 0 y 1, el modelo parece estar bastante seguro de las palabras generadas, ya que las predicciones son coherentes con lo aprendido durante el entrenamiento. Sin embargo, la repetición de frases como "y el mundo albahaca" sugiere que el modelo podría estar atrapado en patrones específicos del entrenamiento.
Por otro lado, en la Oración 2, el modelo es menos seguro. Esta secuencia tiene menos coherencia y contiene palabras inesperadas como "muchos exacto negro", lo que indica que el modelo no está tan seguro de sus predicciones para esta combinación de palabras.
Finalmente, aunque la Oración 3 muestra cierta repetición y puede parecer incoherente ("y el mundo derretida por el mundo peste"), el modelo genera estas repeticiones con bastante confianza. Por lo que el modelo ha aprendido ciertos patrones de repetición del texto de entrenamiento, incluso si la oración generada no es completamente coherente desde un punto de vista semántico.



### Design your own neural network (more layers and different number of neurons)
The goal is to get sentences that make more sense

In [35]:
import torch

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cpu


In [36]:
# Load the text data
file_path = '/content/drive/MyDrive/TALLERES NLP /cap1.txt'
with open(file_path, 'r') as file:
    text = file.read().lower()  # Convert to lowercase for normalization

# Tokenization
import nltk
from nltk.tokenize import TreebankWordTokenizer

nltk.download('punkt')

# Tokenize the text into words
tokenizer = TreebankWordTokenizer()
tokens = tokenizer.tokenize(text)

# Create a vocabulary dictionary
vocab = set(tokens)
word_to_idx = {word: i for i, word in enumerate(vocab)}
idx_to_word = {i: word for word, i in word_to_idx.items()}

# Convert the tokens to numerical indices
indices = [word_to_idx[word] for word in tokens]

# Create sequences for training
sequence_length = 5  # You can experiment with different lengths
X, y = [], []
for i in range(len(indices) - sequence_length):
    X.append(indices[i:i + sequence_length])
    y.append(indices[i + sequence_length])

# Convert X and y to PyTorch tensors and move them to the GPU
X = torch.tensor(X, dtype=torch.long).to(device)
y = torch.tensor(y, dtype=torch.long).to(device)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [37]:
import torch.nn as nn
import torch.optim as optim

class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)  # Embedding layer
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True)  # LSTM layers
        self.fc = nn.Linear(hidden_dim, vocab_size)  # Fully connected layer

    def forward(self, x):
        x = self.embedding(x)  # Convert word indices to embeddings
        lstm_out, _ = self.lstm(x)  # LSTM output
        out = self.fc(lstm_out[:, -1, :])  # Fully connected layer (predict next word)
        return torch.softmax(out, dim=1)  # Softmax for probability distribution

# Set hyperparameters
embedding_dim = 100
hidden_dim = 256
num_layers = 2
vocab_size = len(vocab)

# Instantiate the model and move it to the GPU
model = LSTMModel(vocab_size, embedding_dim, hidden_dim, num_layers).to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [38]:
# Number of epochs
epochs = 60

for epoch in range(epochs):
    model.train()  # Set the model to training mode

    # Forward pass
    outputs = model(X)
    loss = criterion(outputs, y)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 5 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')


Epoch [5/60], Loss: 7.6624
Epoch [10/60], Loss: 7.6624
Epoch [15/60], Loss: 7.6611
Epoch [20/60], Loss: 7.6417
Epoch [25/60], Loss: 7.6060
Epoch [30/60], Loss: 7.5978
Epoch [35/60], Loss: 7.5962
Epoch [40/60], Loss: 7.5956
Epoch [45/60], Loss: 7.5952
Epoch [50/60], Loss: 7.5951
Epoch [55/60], Loss: 7.5949
Epoch [60/60], Loss: 7.5948


In [39]:
def generate_sentence(start_word, model, word_to_idx, idx_to_word, max_length=10):
    model.eval()  # Set the model to evaluation mode
    sentence = [start_word]

    # Convert the start word to an index and move to GPU
    current_word_idx = torch.tensor([word_to_idx[start_word]], dtype=torch.long).unsqueeze(0).to(device)

    for _ in range(max_length - 1):
        with torch.no_grad():
            # Get the model prediction for the next word
            output = model(current_word_idx)

            # Get the word index with the highest probability
            _, predicted_idx = torch.max(output, 1)
            predicted_word = idx_to_word[predicted_idx.item()]

            # Add the predicted word to the sentence
            sentence.append(predicted_word)

            # Update the current word index and move to GPU
            current_word_idx = torch.tensor([word_to_idx[predicted_word]], dtype=torch.long).unsqueeze(0).to(device)

    return ' '.join(sentence)

# Test sentence generation
start_word = 'aureliano'
predicted_sentence = generate_sentence(start_word, model, word_to_idx, idx_to_word, max_length=10)
print(predicted_sentence)


aureliano de de de de de de de de de


In [41]:
# Lista de palabras iniciales
palabras_iniciales = ['aureliano', 'macondo', 'muchos', 'orilla']

# Probar la generación de texto con algunas palabras iniciales
for i, start_word in enumerate(palabras_iniciales):
    predicted_sentence = generate_sentence(start_word, model, word_to_idx, idx_to_word, max_length=5)
    print(f"{i+1}: {predicted_sentence}")

1: aureliano de de de de
2: macondo de de de de
3: muchos de de de de
4: orilla de de de de
